# Building GloVe

In this assignment, you're going to implement GloVe, one of the more popular, effective, and efficient approaches to learning word embeddings. You should use the [paper](http://nlp.stanford.edu/pubs/glove.pdf) for reference, and you're welcome to look to other implementations for guidance as long as the code that you submit is your own and it fits the basic structure provided by the starter code below.

Submit your completed notebook through NYU Classes by 9:30 AM on October 13.

## Setup

We're going to run GloVe on the text of the Stanford Sentiment Treebank (SST) training set. Usually these methods are run on extremely large corpora, but we're using this here to make sure that you can train a reasonable model without waiting for hours or days. 

First, let's load the data as before. For our purposes, we won't need either the labels or any of the test or dev data.

In [2]:
sst_home = '../trees'

import re

def load_sst_data(path):
    data = []
    with open(path) as f:
        for i, line in enumerate(f): 
            example = {}
            text = re.sub(r'\s*(\(\d)|(\))\s*', '', line)
            example['text'] = text[1:]
            data.append(example)
    return data
     
training_set = load_sst_data(sst_home + '/train.txt')

Next let's count cooccurrences on the training set. We'll use a nine-word window. Along the way, we'll also collect a list of all of the index pairs $(i,j)$ that have a count greater than zero. 

To speed up GloVe training below, though, we'll only consider the 250 most frequent words in the corpus.

In [22]:
import collections
import numpy as np

def tokenize(string):
    return string.split()

word_counter = collections.Counter()
for example in training_set:
    word_counter.update(tokenize(example['text']))
vocabulary = [pair[0] for pair in word_counter.most_common()[0:250]]
index_to_word_map = dict(enumerate(vocabulary))
word_to_index_map = dict([(index_to_word_map[index], index) for index in index_to_word_map])

def extract_cooccurrences(dataset, word_map, amount_of_context=4):
    num_words = len(vocabulary)
    cooccurrences = np.zeros((num_words, num_words))
    nonzero_pairs = set()
    for example in dataset:
        words = tokenize(example['text'])
        for target_index in range(len(words)):
            target_word = words[target_index]
            if target_word not in word_to_index_map:
                continue
            target_word_index = word_to_index_map[target_word]
            min_context_index = max(0, target_index - amount_of_context)
            max_word = min(len(words), target_index + amount_of_context)
            for context_index in range(min_context_index, target_index) + range(target_index + 1, max_word):
                context_word = words[context_index]
                if context_word not in word_to_index_map:
                    continue
                context_word_index = word_to_index_map[context_word]
                cooccurrences[target_word_index][context_word_index] += 1.0
                nonzero_pairs.add((target_word_index, context_word_index))
    return cooccurrences, list(nonzero_pairs)
                
cooccurrences, nonzero_pairs = extract_cooccurrences(training_set, vocabulary)
print list(nonzero_pairs)[0]

(238, 115)


## Part 1: Implementation (60%)

### Setting up evalation

To be frank, a GloVe model trained on such a small dataset and vocabulary won't be spectacular, so we won't bother with a full-fledged similarity or analogy evaluation. Instead, we'll use the simple scoring function below, which grades the model on how well it captures ten easy/simple similarity comparisons. The function returns a score between 0 and 10. Random embeddings can be expected to get a score of 5.

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

def similarity(word_one, word_two):
    vec_one = model.get_embedding(word_to_index_map[word_one]).reshape(1, -1)
    vec_two = model.get_embedding(word_to_index_map[word_two]).reshape(1, -1)
    return float(cosine_similarity(vec_one, vec_two))

def score(model):
    score = 0
    score += similarity('a', 'an') > similarity('a', 'documentary')
    score += similarity('in', 'of') > similarity('in', 'picture')
    score += similarity('action', 'thriller') >  similarity('action', 'end')
    score += similarity('films', 'movies') > similarity('films', 'almost')
    score += similarity('film', 'movie') > similarity('film', 'movies')
    score += similarity('script', 'plot') > similarity('script', 'big')
    score += similarity('watch', 'see') > similarity('watch', 'down')
    score += similarity('``', "''") > similarity('``', 'quite')
    score += similarity('funny', 'entertaining') > similarity('funny', 'seems')
    score += similarity('good', 'great') > similarity('good', 'minutes')
    return score

Once you've built and trained the model, you can evaluate it as follows. You may not have to, though, since evaluation is built into the model code below.

In [5]:
score(model)

NameError: name 'model' is not defined

### Defining the model (30%)

**Fill it out to create an implementation of GloVe, then train it on the SST training set.**

Some tips:

- You should use minibatch SGD (the starter code is set up for it), and you should run computation for an entire minibatch at a time using a single `sess.run()` call. This means that many of your variables will have an extra batch dimension in addition to the dimensions that you'd expect from reading the paper. 
- Every time you define any new TF computation, add a comment indicating what shape (/dimensions) you expect the result to be. Use `tf.Print()` and `tf.shape()` to make sure that you're getting what you expect.
- You'll likely need to use `tf.reshape()` and `tf.batch_matmul()` at least once each.
- If you're new to TF, try to find a partner or group to work with. The solution is fairly simple, but finding it may not be.

In [149]:
import tensorflow as tf
import random

class glove:
    def __init__(self, num_words):
        # Define the hyperparameters
        self.dim = 10                # The size of the learned embeddings
        self.alpha = 0.75            # One of the hyperparameters defining the scaling function F
        self.xmax = 50               # One of the hyperparameters defining the scaling function F 
        self.learning_rate = 0.7     # SGD LR - Much higher than you'll see for typical NNs, but it works here
        self.batch_size = 1024       # Somewhat arbitrary - can be tuned, but often tuned for speed, not accuracy
        self.training_epochs = 2100  # This model should train faster per epoch than the logistic regression models,
                                     # so we can afford to run for more epochs. You should feel free to stop the model
                                     # during training if it seems to stop improving.
        self.display_epoch_freq = 25 # How often to test and print out statistics
        self.num_words = num_words   # The number of vectors to learn
        self.embeddings = None       # To be set later
    
        # Define the inputs to the model
        self.target_index = tf.placeholder(tf.int32, shape=[self.batch_size])
        self.context_index = tf.placeholder(tf.int32, shape=[self.batch_size])
        self.cooccurrences = tf.placeholder(tf.float32, shape=[self.batch_size])
        self.count_max = tf.constant([self.xmax], dtype=tf.float32)
        self.scaling_factor = tf.constant([self.alpha], dtype=tf.float32)
        
        # Define the trainable parameters of the model
        self.target_embeddings = tf.Variable(tf.random_normal([self.num_words, self.dim],0,0.1))
        self.context_embeddings = tf.Variable(tf.random_normal([self.num_words, self.dim],0,0.1))
        self.target_biases = tf.Variable(tf.random_normal([self.num_words],0,0.1))
        self.context_biases = tf.Variable(tf.random_normal([self.num_words],0,0.1))
        
        # parallel look-ups for embeddings
        self.target_embedding = tf.nn.embedding_lookup([self.target_embeddings], self.target_index)
        self.context_embedding = tf.nn.embedding_lookup([self.context_embeddings], self.context_index)
        self.target_bias = tf.nn.embedding_lookup([self.target_biases], self.target_index)
        self.context_bias = tf.nn.embedding_lookup([self.context_biases], self.context_index)
        self.F = tf.minimum(1.0, tf.pow(tf.div(self.cooccurrences, self.count_max), 
                                                       self.scaling_factor))
        
        # Define the forward computation of the model
        # The final result that you compute should be a 1024-dimensional vector of example-by-example 
        # cost function values called `self.example_cost`.
        self.target_context_mult = tf.reduce_sum(tf.mul(self.target_embedding, 
                                                        self.context_embedding), 1) # sum over axis 1
        self.square = tf.square(tf.add_n([self.target_context_mult, 
                                          self.target_bias, 
                                          self.context_bias, 
                                          tf.neg(tf.log(tf.to_float(self.cooccurrences)))]))

        self.example_cost = tf.mul(self.F, self.square)
        # Define the cost function
        self.total_cost = tf.reduce_mean(self.example_cost)
        
        # This library call performs the main SGD update equation
        self.optimizer = tf.train.GradientDescentOptimizer(self.learning_rate).minimize(self.total_cost)
        self.embeddings = tf.add(self.target_embeddings, self.context_embeddings)
        
        # Create an operation to fill zero values in for W and b
        self.init = tf.initialize_all_variables()
        
        # Initialize the model
        self.sess = tf.Session()
        
        self.sess.run(self.init)
        
    def train(self, cooccurrences, nonzero_pairs, num_words):
        self.embeddings = None  # If we restart training, make sure to clear the cached embeddings
        print 'Training.'
        
        # Training cycle - In one epoch, we'll visit each nonzero entry in cooccurrences once
        for epoch in range(self.training_epochs):
            random.shuffle(nonzero_pairs)

            avg_cost = 0.
            total_batches = int(len(nonzero_pairs) / self.batch_size)
            
            # Loop over all batches in epoch
            for i in range(total_batches):
                # Assemble a minibatch dictionary to feed to `sess.run()`
                feed = {}
                # randomly sample self.batch_size examples from the data
                pairs = list(nonzero_pairs)[i*self.batch_size: (i+1)*self.batch_size]
                target_indexes = []
                context_indexes = []
                cooccur = []
                for pair in pairs:
                    target_indexes.append(pair[0])
                    context_indexes.append(pair[1])
                    cooccur.append(cooccurrences[pair[0]][pair[1]])
                feed = {self.target_index: target_indexes, 
                        self.context_index: context_indexes, 
                        self.cooccurrences: cooccur}
                # Run the optimizer to take a gradient step, and also fetch the value of the 
                # cost function for logging
                _, c = self.sess.run([self.optimizer, self.total_cost], 
                                     feed_dict=feed)                                    
                # Compute average loss
                avg_cost += c / total_batches
                
            # Display some statistics about the step
            if (epoch+1) % self.display_epoch_freq == 0:
                self.cache_embeddings()  # Make sure we run scoring with a fresh copy of the embeddings
                print "Epoch:", (epoch+1), "Cost:", avg_cost, "Score:", score(self)

    def cache_embeddings(self):
        # Fill in self.embeddings with a matrix (in NumPy format) containing one vector per word, 
        # representing the final output of GloVe. It should be possible to index into that
        # matrix using `get_embedding` below.
        # self.embeddings = np.zeros((self.num_words, self.dim))
        
        self.embeddings = tf.add(self.target_embeddings, self.context_embeddings)
        
        
    def get_embedding(self, index):
        if self.embeddings is None:
            cache_embeddings()
        return self.embeddings.eval(session=self.sess)[index, :]

### Training a working model (30%)

You should use the following commands to train the model for at least 2000 steps. If your model works, it will usually converge to a score of 10 within that many steps.

Tips:

- You cannot run this until you've completed the previous code block. That's because the starter code doesn't actually define at trainable model.
- The score and cost for the first few hundred epochs of training will vary quite a bit due to the random initialization. The real measure of success is how the model does once it nears convergence.
- Make sure to show the full output for a real run in your notebook when you submit. I will not retrain your model to grade it.

In [150]:
model = glove(len(vocabulary))  # Create the model

In [151]:
model.train(cooccurrences, nonzero_pairs, len(vocabulary))  # Train it

Training.
Epoch: 25 Cost: 0.100168309428 Score: 2
Epoch: 50 Cost: 0.0944384020386 Score: 4
Epoch: 75 Cost: 0.0924470885233 Score: 4
Epoch: 100 Cost: 0.0912781556447 Score: 4
Epoch: 125 Cost: 0.0892373210553 Score: 7
Epoch: 150 Cost: 0.0865125671932 Score: 7
Epoch: 175 Cost: 0.0833432003856 Score: 7
Epoch: 200 Cost: 0.0801719409047 Score: 9
Epoch: 225 Cost: 0.0770528486958 Score: 10
Epoch: 250 Cost: 0.0743092187878 Score: 10
Epoch: 275 Cost: 0.0717547635237 Score: 10
Epoch: 300 Cost: 0.0693541609428 Score: 10
Epoch: 325 Cost: 0.0670668171211 Score: 10
Epoch: 350 Cost: 0.0651142620905 Score: 10
Epoch: 375 Cost: 0.063206298559 Score: 10
Epoch: 400 Cost: 0.0615662512454 Score: 10
Epoch: 425 Cost: 0.0600534141289 Score: 10
Epoch: 450 Cost: 0.0589321727554 Score: 10
Epoch: 475 Cost: 0.0578365049353 Score: 10
Epoch: 500 Cost: 0.0567841970108 Score: 10
Epoch: 525 Cost: 0.0558440400796 Score: 10
Epoch: 550 Cost: 0.0550860566172 Score: 10
Epoch: 575 Cost: 0.0544035652596 Score: 10
Epoch: 600 Cos

## Part 2: Questions (40%)

Fill in your answers below.

**Q1:** What do the entries on the diagonal of the `cooccurrences` matrix represent?

**Answer:** The diagonal entries of the matrix represent how many times a word appears withint the context of itself.

**Q2:** Deleting the weighting function $F$ should hurt the performance of the model. Why would you expect this to be the case?

**Answer:** If the function $F$ is deleted, then it means that all of the cooccurrences are weighed equally. If some of the cooccurrences are very rare, then they actually give little information for our task here so they can be regarded as noise in some sense. So it's better to mitigate the effect of the noise by giving them smaller weights.

**Q3:** What would you expect to happen if you used a learning rate of 0.0001. Why? (You're welcome to try it, but that shouldn't be the sole basis for your answer.)

**Answer:** The convergence rate will be very slow, because the step size for gradient descent now is very small. But sometimes it might be possible that it will eventually give a better optimality than much larger rate like 1.0 even though it will take a much longer time. 

**Q4:** If you used 100 times more training data, would the model take 100 times as long to train? Just as long as now? Somewhere in between? Choose one and (informally) defend your answer.

**Answer:** The running time will become somewhere in between. When we have 100 times more training data, we can expect that the variety of the data is greater than before, so the direction of gradient will be more "accurate" in the sense that it will be a better approximate of the global gradient. Also, with more data, the estimate of co-occurrence matrix will be more accurate as well according to the "law of large numbers".